In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pickle

## LOADING THE DATA

In [2]:
signature_data = []
with open("./data/abus/signature.pkl", "rb") as sig_file:
    signature_data = pickle.load(sig_file)
    
for i in range(len(signature_data)):
    x, y = signature_data[i]
    y=np.array(y, dtype=np.float32)
    x = torch.tensor(x)
    y = torch.tensor(y).unsqueeze(0)
    signature_data[i] = (x,y)

In [3]:
x,y = signature_data[0]

print(x.shape, y.shape)

torch.Size([40, 360]) torch.Size([1])


## Normalize

In [4]:
normalized_signature = []

for sample in signature_data:    
    x, y = sample
    for i in range(40):
        s = x[i,:]    
        if s.max() != 0:
            s = (torch.abs(s - torch.mean(s)))/torch.sqrt(torch.var(s))
        x[i,:] = s
        
    normalized_signature.append((x,y))

print(len(normalized_signature))

60


In [5]:
x,y = normalized_signature[0]
print(x)

tensor([[1.3452, 1.3452, 1.3452,  ..., 1.3452, 1.3452, 1.3452],
        [0.9370, 0.9370, 0.9370,  ..., 0.9370, 0.9370, 0.9370],
        [0.9113, 0.9113, 0.9113,  ..., 0.9113, 0.9113, 0.9113],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [6]:
class SequenceClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device='cpu'):
        super(SequenceClassifier, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size * 40, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [7]:
num_epochs = 30
learning_rate = 10e-4
device = 'cuda'
criterion = torch.nn.CrossEntropyLoss()

In [8]:
signature_data = normalized_signature
num_correct_predictions = 0
num_samples = len(signature_data)
main_loop = tqdm(range(len(signature_data)))
for idx in main_loop:
    train_data = signature_data[:idx] + signature_data[idx+1:]
    model = SequenceClassifier(360, 40, 1, 2, device=device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    test_sample = signature_data[idx]
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, num_workers=8, shuffle=True, pin_memory=True)
    for epoch in range(num_epochs):
        for data in train_loader:
            
            x, y = data
            x = x.to(device).squeeze(1)
            y = y.to(device).squeeze(1).long()  
                    
            predictions = model(x)
            predictions = torch.nn.functional.softmax(predictions)
            loss = criterion(predictions, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
                
        total = 0
        num_corrects = 0
       
    for data in train_loader:
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            predictions = torch.nn.functional.softmax(predictions)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
    
    print(f"Testing sample #{idx}: train acc is: {num_corrects}/{total}")
                        
    x, y = test_sample
    x = x.to(device).unsqueeze(0)
    y = y.to(device).unsqueeze(0).long()  

    model.eval()

    with torch.no_grad():

        predictions = model(x)
        predictions = torch.nn.functional.softmax(predictions)
        _, predictions = predictions.max(1)
        num_correct_predictions += (predictions == y).float().sum()
        
    model.train()
    main_loop.set_postfix(test_accuracy=(num_correct_predictions).item()) 
        
print(f"Finished: test acc is: {num_correct_predictions}/{num_samples}")

  0%|          | 0/60 [00:00<?, ?it/s]

/tmp/ipykernel_147537/704394107.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.nn.functional.softmax(predictions)


Testing sample #0: train acc is: 59.0/59


/tmp/ipykernel_147537/704394107.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.nn.functional.softmax(predictions)
/tmp/ipykernel_147537/704394107.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.nn.functional.softmax(predictions)


Testing sample #1: train acc is: 59.0/59
Testing sample #2: train acc is: 56.0/59
Testing sample #3: train acc is: 54.0/59
Testing sample #4: train acc is: 56.0/59
Testing sample #5: train acc is: 52.0/59
Testing sample #6: train acc is: 59.0/59
Testing sample #7: train acc is: 58.0/59
Testing sample #8: train acc is: 59.0/59
Testing sample #9: train acc is: 59.0/59
Testing sample #10: train acc is: 59.0/59
Testing sample #11: train acc is: 55.0/59
Testing sample #12: train acc is: 59.0/59
